<a href="https://colab.research.google.com/github/JoshMcConkie/control-lab/blob/main/cruise_control/notebooks/BasicPIDCruiseControl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

###Frictionless System

Manual

In [ ]:
def pid_simulation_steps(Kp=1.0, Ki=0.0, Kd=0.0):
  x_target_1 = 100
  x_target_2 = 50
  x_target_3 = 200
  noise = np.random.normal(0, 1, 100)

  target_1_time = 0
  target_2_time = 0
  target_3_time = 0
  target = x_target_1

  x = 0
  v = 0

  time_step = .1
  T=30
  N = int(T/time_step)

  error_1 = 0
  error_sum = 0
  de_prev = 0

  x_hist = []
  v_hist = []
  a_hist = []
  time_hist = []
  error_hist = []
  P_hist = []
  I_hist = []
  D_hist = []

  for i in range(N):

    if i == N//3: # introduce a sudden change in x target
      target = x_target_2
    elif i == 2*N//3:
      target = x_target_3
    t = i*time_step

    x_meas = x + np.random.normal(0, .5) # introduce measurement noise
    error = target - x_meas

# Moving Average process (low pass) for derivative to smooth out error spikes
    a = .8 # for smoothing
    de_raw = (error - error_1)/time_step
    de = a * de_prev + (1-a)*de_raw
    de_prev = de

    P = Kp * error
    I = Ki * error_sum
    D = Kd * de
    P_hist.append(P)
    I_hist.append(I)
    D_hist.append(D)
# Anti-Windup in integration
    max_accel = 10
    a_unsat = P + I + D
    a = max(min(a_unsat, max_accel), -max_accel)

    if abs(a_unsat) < max_accel:
      error_sum += error*time_step

    v += a * time_step
    x += v * time_step

    error_1 = error

    x_hist.append(x)
    v_hist.append(v)
    a_hist.append(a)
    time_hist.append(t)
    error_hist.append(error)

    if (target == x_target_1) and (target_1_time == 0):
      if abs(x - x_target_1) < .05 * x_target_1:
        target_1_time = t
    elif (target == x_target_2) and (target_2_time == 0):
      if abs(x - x_target_2) < .05 * x_target_2:
        target_2_time = t
    elif (target == x_target_3) and (target_3_time) == 0:
      if abs(x - x_target_3) < .05 * x_target_3:
        target_3_time = t

  print("Target 1 time: ", target_1_time)
  print("Target 2 time: ", target_2_time)
  print("Target 3 time: ", target_3_time)
  plt.figure(figsize=(12, 12))
  plt.subplot(4, 1, 1)
  plt.plot(time_hist, x_hist, label="Position")
  plt.axhline(x_target_1, color='g', linestyle='--', label="Target")
  plt.axhline(x_target_2, color='b', linestyle='--', label="Target 2")
  plt.axhline(x_target_3, color='r', linestyle='--', label="Target 3")
  plt.axvline(target_1_time, color='g', linestyle='--', label="Target 1 +- 5%")
  plt.axvline(target_2_time, color='b', linestyle='--', label="Target 2 +- 5%")
  plt.axvline(target_3_time, color='r', linestyle='--', label="Target 3 +- 5%")
  plt.ylabel("Position")
  plt.legend()

  plt.subplot(4, 1, 2)
  plt.plot(time_hist, v_hist, label="Velocity", color='g')
  plt.ylabel("Velocity")
  plt.legend()

  plt.subplot(4, 1, 3)
  plt.plot(time_hist, error_hist, label="Error", color='orange')
  plt.xlabel("Time (s)")
  plt.ylabel("Error")
  plt.legend()

  plt.subplot(4, 1, 4)
  plt.plot(time_hist, P_hist, label="P", color='pink')
  plt.plot(time_hist, I_hist, label="I", color='violet')
  plt.plot(time_hist, D_hist, label="D", color='orange')
  plt.xlabel("Time (s)")
  plt.ylabel("Value")
  plt.legend()

  plt.tight_layout()
  plt.show()


  return target_1_time, target_2_time, target_3_time, x_target_1, x_target_2, x_target_3

interact(pid_simulation_steps,
  Kp=FloatSlider(value=.30, min=0.0, max=1.0, step=0.1, description='Kp'),
  Ki=FloatSlider(value=0.0, min=0.0, max=1.0, step=0.02, description='Ki'),
  Kd=FloatSlider(value=.75, min=0.0, max=1.0, step=0.05, description='Kd')
)


In [ ]:
def pid_simulation(Kp=1.0, Ki=0.0, Kd=0.0):
  noise = np.random.normal(0, 1, 100)

  target_time = 0
  target = 100

  x = 0
  v = 0

  time_step = .1
  T=30
  N = int(T/time_step)

  error_1 = 0
  error_sum = 0
  de_prev = 0

  x_hist = []
  v_hist = []
  a_hist = []
  time_hist = []
  error_hist = []
  P_hist = []
  I_hist = []
  D_hist = []

  for i in range(N):
    t = i*time_step

    x_meas = x + np.random.normal(0, .5) # introduce measurement noise
    error = target - x_meas

# Moving Average process (low pass) for derivative to smooth out error spikes
    a = .8 # for smoothing
    de_raw = (error - error_1)/time_step
    de = a * de_prev + (1-a)*de_raw
    de_prev = de

    P = Kp * error
    I = Ki * error_sum
    D = Kd * de
    P_hist.append(P)
    I_hist.append(I)
    D_hist.append(D)
# Anti-Windup in integration
    max_accel = 10
    a_unsat = P + I + D
    a = max(min(a_unsat, max_accel), -max_accel)

    if abs(a_unsat) < max_accel:
      error_sum += error*time_step

    v += a * time_step
    x += v * time_step

    error_1 = error

    x_hist.append(x)
    v_hist.append(v)
    a_hist.append(a)
    time_hist.append(t)
    error_hist.append(error)

    if (target_time == 0):
      if abs(x - target) < .05 * target:
        target_time = t

  print("Target time: ", target_time)

  plt.figure(figsize=(12, 12))
  plt.subplot(4, 1, 1)
  plt.plot(time_hist, x_hist, label="Position")
  plt.axhline(target, color='g', linestyle='--', label="Target")
  plt.axvline(target_time, color='g', linestyle='--', label="Target +- 5%")

  plt.ylabel("Position")
  plt.legend()

  plt.subplot(4, 1, 2)
  plt.plot(time_hist, v_hist, label="Velocity", color='g')
  plt.ylabel("Velocity")
  plt.legend()

  plt.subplot(4, 1, 3)
  plt.plot(time_hist, error_hist, label="Error", color='orange')
  plt.xlabel("Time (s)")
  plt.ylabel("Error")
  plt.legend()

  plt.subplot(4, 1, 4)
  plt.plot(time_hist, P_hist, label="P", color='pink')
  plt.plot(time_hist, I_hist, label="I", color='violet')
  plt.plot(time_hist, D_hist, label="D", color='orange')
  plt.xlabel("Time (s)")
  plt.ylabel("Value")
  plt.legend()

  plt.tight_layout()
  plt.show()

interact(pid_simulation,
  Kp=FloatSlider(value=.30, min=0.0, max=1.0, step=0.1, description='Kp'),
  Ki=FloatSlider(value=0.0, min=0.0, max=1.0, step=0.02, description='Ki'),
  Kd=FloatSlider(value=.75, min=0.0, max=1.0, step=0.05, description='Kd')
)


**Optimized**

Penalties:
- The amount of time after reaching target spent outside close proximity to the desired outcome
- Total absolute acceleration after reaching threshold and before new mission
- Time to target

In [ ]:
from scipy.optimize import differential_evolution

def run_pid_sim(Kp=1.0, Ki=0.0, Kd=0.0, x_target=100):
  np.random.seed(42)
  t_to_target = None
  x = 0
  v = 0
  time_step = .1
  T=10
  N = int(T/time_step)
  error_1 = 0
  error_sum = 0
  de_prev = 0

  post_target_error = []
  post_target_acceleration = []
  for i in range(N):
    t = i*time_step

    x_meas = x + np.random.normal(0, .5) # introduce measurement noise
    error = x_target - x_meas


# Moving Average process (low pass) for derivative to smooth out error spikes
    a = .8 # for smoothing
    de_raw = (error - error_1)/time_step
    de = a * de_prev + (1-a)*de_raw
    de_prev = de

    P = Kp * error
    I = Ki * error_sum
    D = Kd * de

# Anti-Windup in integration
    max_accel = 10
    a_unsat = P + I + D
    a = max(min(a_unsat, max_accel), -max_accel)

    if abs(a_unsat) < max_accel:
      error_sum += error*time_step

    v += a * time_step
    x += v * time_step

    error_1 = error
    if t_to_target is not None:
          post_target_error.append(error)
          post_target_acceleration.append(a)
    if t_to_target is None:
      if abs(x - x_target) < .05 * x_target:
        t_to_target = t
  if t_to_target is None:
    return T + 10, 1000, 1000, 1000

  if not post_target_error:
    post_target_variance = 0.0
  else:
      post_target_variance = np.var(post_target_error, ddof=0)

  if len(post_target_acceleration) == 0:
    post_target_tot_accel = 0  # or some other appropriate value
  else:
    post_target_tot_accel = sum(abs(a) * time_step for a in post_target_acceleration)/len(post_target_acceleration)


  # normalize outcomes

  return t_to_target/T, post_target_variance, post_target_tot_accel/max_accel, t_to_target

def cost(params, w1=20, w2=1, w3=10):
  Kp, Ki, Kd = params
  quickness, post_target_variance, oscillation,t_to_target = run_pid_sim(Kp, Ki, Kd)
  return quickness*w1 + post_target_variance*w2 + oscillation*w3

Kp_bounds = (0,1)
Ki_bounds = (0,.5)
Kd_bounds = (.5,2)
bounds = [Kp_bounds, Ki_bounds, Kd_bounds]

optimized = differential_evolution(cost, bounds)
print(optimized.x)




In [ ]:
pid_simulation(*optimized.x)

In [ ]:
import pandas as pd
import itertools
from tqdm import tqdm
from functools import partial

#  Top‐level cost function (picklable)
def weighted_cost(params, w1, w2, w3):
    q, var, osc, t_to_target = run_pid_sim(*params)
    return w1*q + w2*var + w3*osc

records = []
weights = np.linspace(0.5, 5, 20)
for w1, w2, w3 in tqdm(itertools.product(weights, repeat=3),total=len(weights)**3,desc="Weight Sweep"):
  cost_fn = partial(weighted_cost, w1=w1, w2=w2, w3=w3)
  optimized = differential_evolution(cost_fn, bounds, disp=False)
  Kp, Ki, Kd = optimized.x
  q, var, osc, tt_target = run_pid_sim(Kp, Ki, Kd)
  records.append({
        "w_quickness": w1,
        "w_variance":  w2,
        "w_oscillation": w3,
        "Kp_opt":      Kp,
        "Ki_opt":      Ki,
        "Kd_opt":      Kd,
        "quickness": q,
        "post_variance":  var,
        "post_oscillation": osc,
        "cost":        optimized.fun,
        "t_to_target": tt_target
    })

df_opt_outcomes = pd.DataFrame(records)
df_opt_outcomes.sort_values(by="cost")

In [ ]:

df_opt_outcomes['tot_w'] = df_opt_outcomes['w_quickness'] + df_opt_outcomes['w_variance'] + df_opt_outcomes['w_oscillation']
df_opt_outcomes['wr_quickness'] = df_opt_outcomes['w_quickness']/df_opt_outcomes['tot_w']
df_opt_outcomes['wr_variance'] = df_opt_outcomes['w_variance']/df_opt_outcomes['tot_w']
df_opt_outcomes['wr_oscillation'] = df_opt_outcomes['w_oscillation']/df_opt_outcomes['tot_w']


# Graph quickness against its proportional weight
df_opt_outcomes.tail(15)

In [ ]:
""

# print(f"weights: {weights}")
# print("Kp: ", Kp)
# print("Ki: ", Ki)
# print("Kd: ", Kd)
# print("Quickness: ", quickness)
# print("Post target variance: ", post_target_variance)
# print("Oscillation: ", oscillation)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df_opt_outcomes["t_to_target"], df_opt_outcomes["post_variance"], alpha=.6,s=100)
plt.xlabel("Quickness")
plt.ylabel("Variance")
plt.title("Variance vs Quickness")
plt.grid(True)
plt.show()
